In [1]:
# load tqdm
# !pip3 install --force https://github.com/chengs/tqdm/archive/colab.zip

In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
df = pd.read_csv(
    '../Data/smile-annotations-final.csv',
    names=['id','text','category'])

In [4]:
df.set_index('id', inplace=True)

In [5]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [7]:
df = df[~df.category.str.contains('\|')]

In [8]:
df = df[df.category != 'nocode']

In [9]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [10]:
label_dict = {l:i for i,l in enumerate(df.category.unique())}
num2label = {v:k for k, v in label_dict.items()}

In [11]:
num2label

{0: 'happy',
 1: 'not-relevant',
 2: 'angry',
 3: 'disgust',
 4: 'sad',
 5: 'surprise'}

In [12]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [13]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
SEED = 42069

In [16]:
Xt, Xv, yt, yv = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=SEED,
    stratify=df.label.values
)

In [17]:
df['type'] = ['tmp'] * df.shape[0]

In [18]:
df.head()

,text,category,label,type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,tmp
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,tmp
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,tmp
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,tmp
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,tmp


In [19]:
df.loc[Xt, 'type'] = 'train'
df.loc[Xv, 'type'] = 'val'

In [20]:
df.groupby(['category', 'label', 'type']).count()

text
category     label type       
angry        2     train    48
                   val       9
disgust      3     train     5
                   val       1
happy        0     train   966
                   val     171
not-relevant 1     train   182
                   val      32
sad          4     train    27
                   val       5
surprise     5     train    30
                   val       5

In [21]:
!pip install transformers >> /dev/null

In [22]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [23]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [24]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/anbang/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [25]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.type=='train'].label.values)

In [46]:
print(input_ids_train[0])
print(input_ids_train[1])
print(attention_masks_train[0])
print(labels_train[0])

tensor([  101, 16092,  3897,  2007, 10098, 18982,  1001,  2293, 10105,  2015,
         1006,  2013,  1030,  2329,  7606, 14820,  8299,  1024,  1013,  1013,
         1056,  1012,  2522,  1013,  1053,  2549,  2595, 26760,  2140,  2692,
         2229,  2226,  1007,  8299,  1024,  1013,  1013,  1056,  1012,  2522,
         1013,  1044,  2692,  6777, 19279,  2497, 13088,  4160,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [26]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.type=='val'].label.values)

In [27]:
train_ds = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_ds = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [28]:
print(len(train_ds), len(val_ds))

1258 223


In [40]:
from transformers import BertForSequenceClassification,BertPreTrainedModel,BertConfig,BertModel

In [38]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [170]:
import torch
import torch.nn as nn

from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import gensim.downloader

In [175]:
glove_vectors = gensim.downloader.load('glove-twitter-25')
keys = ['happy','objective','angry','disgust','sad','surprise']
emotion_space = torch.tensor([glove_vectors[key] for key in keys]).type(torch.LongTensor)
print(emotion_space)

tensor([[-1,  0,  0,  0,  0,  0,  2,  1, -1,  0,  0,  0, -3,  0,  0,  0,  0, -1,
          0,  0,  0,  0, -1,  0,  0],
        [ 0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0, -2,  0,  0,  0,  0,  1,
          0,  0,  0,  0,  0,  0, -1],
        [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0, -2,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0, -2, -1,  0,  0,  0, -1,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0, -1,  0, -4,  0,  0,  0,  0,  0,
          0,  0,  0,  1,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0, -2,  0,  0,  1,  0,  0,
         -1,  0,  0,  0,  0,  0,  0]])


In [498]:
emotion_space = emotion_space.cuda()

In [499]:
class BertForSequenceClassification(BertPreTrainedModel):
    """
参数：
    config：指定的bert模型的预训练参数
    num_labels：分类的类别数量
输入：
    input_ids：训练集，torch.LongTensor类型，shape是[batch_size, sequence_length]
    token_type_ids：可选项，当训练集是两句话时才有的。
    attention_mask：可选项，当使用mask才有，可参考原论文。
    labels：数据标签，torch.LongTensor类型，shape是[batch_size]
输出：
    如果labels不是None（训练时）：输出的是分类的交叉熵
    如果labels是None（评价时）：输出的是shape为[batch_size, num_labels]估计值
    
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
    num_labels = 2
    model = BertForSequenceClassification(config, num_labels)
    logits = model(input_ids, token_type_ids, input_mask)
    """
    def __init__(self, config=None, num_labels=2, label_dim=25):
        if not config:
            config = BertConfig(hidden_size=768,
                        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
        super(BertForSequenceClassification, self).__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.header = nn.Linear(config.hidden_size, label_dim)
        self.loss_fct = torch.nn.CrossEntropyLoss()
        #self.apply(super(BertForSequenceClassification, self).init_bert_weights)

    def forward(self, input_ids, label_encoding, token_type_ids=None, attention_mask=None, labels=None):
        try:
            _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, return_dict=False)
        except Exception:
            print(input_ids,labels)
        pooled_output = self.dropout(pooled_output)
        pooled_output = self.header(pooled_output)
        pooled_output = torch.tile(pooled_output.unsqueeze(1), (1,label_encoding.size()[0],1))
        logits = torch.sum(pooled_output * label_encoding, axis=-1)
        #print(logits.size())
        if labels is not None:
            #print(input_ids.size(), labels)
            loss =self.loss_fct(logits, labels.view(-1))
            return loss,logits
        else:
            return logits

In [500]:
'''
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)
'''
model = BertForSequenceClassification().from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [501]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [502]:
BATCH_SIZE = 2
EPOCHS = 10

In [503]:
train_dl = DataLoader(
    train_ds,
    sampler=RandomSampler(train_ds),
    batch_size=BATCH_SIZE
)

val_dl = DataLoader(
    val_ds,
    sampler=RandomSampler(val_ds),
    batch_size=BATCH_SIZE
)

In [504]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [505]:
optim = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

In [506]:
scheduler = get_linear_schedule_with_warmup(
    optim,
    num_warmup_steps=0,
    num_training_steps=len(train_dl)*EPOCHS
)

 Defining Performance Metrics

In [507]:
import numpy as np
from sklearn.metrics import f1_score

In [508]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [509]:
def print_acc_per_class(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    for label in np.unique(labels_flat):
        total_true = preds_flat[labels_flat == label]
        true_positives = total_true[total_true == label]
        print(f'Class: {num2label[label]}')
        print(f'Accuracy: {len(true_positives)}/{len(total_true)}')
    

In [510]:
import random

In [511]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [512]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [517]:
def evaluate(model, val_dl):
    model.eval()
    val_loss = 0
    predictions, true_vals = [], []
    for batch in tqdm(val_dl):
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'label_encoding':emotion_space,
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        val_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    val_loss_avg = val_loss/len(val_dl)
    prediction = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    return val_loss_avg, predictions, true_vals


In [518]:
!mkdir Models

mkdir: 无法创建目录"Models": 文件已存在


In [519]:
max_v = -10000
min_v = 10000

def fit(model, epochs, train_dl, val_dl, optimizer):
    global max_v, min_v
    for epoch in range(epochs):
        model.train()
        training_loss = 0
        tqdm_progress = tqdm(
            train_dl,
            desc=f'Epoch {epoch+1}',
            leave=False,
            disable=False
            )
        for batch in tqdm_progress:
            model.zero_grad()
            '''
            max_v = max(max_v,torch.max(batch[0]).data)
            min_v = min(min_v,torch.min(batch[0]).data)
        break
            '''
            
            batch = tuple(b.to(device) for b in batch)
            inputs = {
                'input_ids' : batch[0],
                'label_encoding' : emotion_space,
                'attention_mask' : batch[1],
                'labels' : batch[2]
            }
            outputs = model(**inputs)
            loss = outputs[0]
            training_loss += loss.data
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            tqdm_progress.set_postfix({'training_loss': f'{loss.item()/len(batch):.3f}'})

            
        
        torch.save(model.state_dict(), f'./Models/BERT_ft_epoch{epoch}.model')
        tqdm.write(f'Epoch {epoch}')
        training_loss_avg = training_loss/len(train_dl)
        tqdm.write(f'Training Loss: {training_loss_avg}')

        val_loss, val_preds, val_true = evaluate(model, val_dl)
        val_preds = np.vstack(np.array(val_preds))
        val_f1 = f1_score_func(val_preds, val_true)
        tqdm.write(f'Val Loss: {val_loss}')
        tqdm.write(f'F1 Score (weighted): {val_f1}')


In [520]:
fit(model, EPOCHS, train_dl, val_dl, optim)
print(max_v ,min_v)

Epoch 1:   0%|          | 0/629 [00:00<?, ?it/s]

Epoch 0
Training Loss: 0.6164447665214539


  0%|          | 0/112 [00:00<?, ?it/s]

Val Loss: 0.6064035602012154
F1 Score (weighted): 0.8502595055703952


Epoch 2:   0%|          | 0/629 [00:00<?, ?it/s]

...

Val Loss: 0.8663596970115667
F1 Score (weighted): 0.8892680325713841


Epoch 10:   0%|          | 0/629 [00:00<?, ?it/s]

Epoch 9
Training Loss: 0.007470006588846445


  0%|          | 0/112 [00:00<?, ?it/s]

Val Loss: 0.8663591125294821
F1 Score (weighted): 0.8892680325713841
-10000 10000


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!cp Models/BERT_ft_epoch9.model /gdrive/My\ Drive/

In [ ]:
model.load_state_dict(
    torch.load('Models/BERT_ft_epoch9.model')
)

In [ ]:
_, predictions, true_vals = evaluate(model, val_dl)

In [ ]:
print_acc_per_class(np.vstack(predictions), true_vals)